<h2><center> Convolutional Neural Network Model -Deep Neuron AI </center></h2>

In [1]:
import torch

In [2]:
torch.cuda.device(0)

In [3]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080 Laptop GPU'

In [4]:
torch.cuda.is_available()

True

In [5]:
num_epochs = 5 
batch_size = 100
learning_rate = 0.001

In [6]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
train_dataset = MNIST(root = './', train=True, download=True, transform=transforms.ToTensor())
test_dataset= MNIST(root = './', train=False, download=True, transform=transforms.ToTensor())

100.0%


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw



100.0%


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw



23.8%

100.0%


Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw



100.0%

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw



In [7]:
import torch.utils.data as Data
train_loader = Data.DataLoader(dataset=train_dataset, 
                               batch_size=batch_size, 
                               shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset, 
                               batch_size=batch_size, 
                               shuffle=False) # shuffle=False

In [8]:
import torch.nn as nn

In [9]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(stride=2, kernel_size=2)
    )
    self.dense = nn.Sequential(
        nn.Linear(in_features=14*14*128, out_features=1024),
        nn.ReLU(),
        nn.Linear(1024, 10)
    )

  def forward(self, x):
    out = self.conv(x)
    out = out.view(-1, 14*14*128)
    out = self.dense(out)
    return out

In [10]:
model = Model()
model.cuda()

Model(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

In [11]:
import torch.optim as opt
from torch.autograd import Variable

In [12]:
loss_func = nn.CrossEntropyLoss()
optimizer = opt.Adam(model.parameters(), lr=learning_rate)

In [13]:
for epoch in range(num_epochs):
  for idx, (images, labels) in enumerate(train_loader):
    images = Variable(images.type(torch.cuda.FloatTensor))
    labels = Variable(labels.type(torch.cuda.LongTensor))

    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_func(outputs, labels)
    loss.backward()
    optimizer.step()

    if (idx+1)%100 == 0:
      print("Epoch: %d, Batch: %d, Loss: %.4f" %(epoch+1, idx+1, loss.data))

Epoch: 1, Batch: 100, Loss: 0.0677
Epoch: 1, Batch: 200, Loss: 0.0566
Epoch: 1, Batch: 300, Loss: 0.0507
Epoch: 1, Batch: 400, Loss: 0.0987
Epoch: 1, Batch: 500, Loss: 0.0570
Epoch: 1, Batch: 600, Loss: 0.0174
Epoch: 2, Batch: 100, Loss: 0.0104
Epoch: 2, Batch: 200, Loss: 0.0149
Epoch: 2, Batch: 300, Loss: 0.0043
Epoch: 2, Batch: 400, Loss: 0.0053
Epoch: 2, Batch: 500, Loss: 0.0148
Epoch: 2, Batch: 600, Loss: 0.0364
Epoch: 3, Batch: 100, Loss: 0.0052
Epoch: 3, Batch: 200, Loss: 0.0277
Epoch: 3, Batch: 300, Loss: 0.0129
Epoch: 3, Batch: 400, Loss: 0.0304
Epoch: 3, Batch: 500, Loss: 0.0092
Epoch: 3, Batch: 600, Loss: 0.1536
Epoch: 4, Batch: 100, Loss: 0.0121
Epoch: 4, Batch: 200, Loss: 0.0006
Epoch: 4, Batch: 300, Loss: 0.0417
Epoch: 4, Batch: 400, Loss: 0.0014
Epoch: 4, Batch: 500, Loss: 0.0175
Epoch: 4, Batch: 600, Loss: 0.0234
Epoch: 5, Batch: 100, Loss: 0.0010
Epoch: 5, Batch: 200, Loss: 0.0129
Epoch: 5, Batch: 300, Loss: 0.0101
Epoch: 5, Batch: 400, Loss: 0.0016
Epoch: 5, Batch: 500

In [14]:
correct = 0
total = 0
for images, labels in test_loader:
  images = Variable(images.type(torch.cuda.FloatTensor))
  outputs = model(images)
  _, pred = torch.max(outputs.data, 1) # 1=(from) row
  
  correct += (pred == labels.type(torch.cuda.LongTensor)).sum() # 猜對量sum
  total += labels.size(0)

print('Accuracy:%.3f%%' %(100.0 * float(correct)/float(total)))

Accuracy:98.590%
